In [2]:
import feedparser
import DCDSFunctions as f
import pandas as pd
import pickle
import datetime

<b> Scrape records of application for H1B visa applications from h1bdata.info </b>

In [3]:
# H1BDATA.info
import requests
from bs4 import BeautifulSoup
import time
import random

In [4]:
# Sample query
years = list(range(2014,2020))
years = [str(y) for y in years]
years

['2014', '2015', '2016', '2017', '2018', '2019']

In [71]:
# Scrape Job Title Information

job_title_url = 'https://h1bdata.info/topjobs.php'
title_resp = requests.get(job_title_url)

titles_all = BeautifulSoup(title_resp.content, 'html.parser') # soup type
title_block = titles_all.find('tbody') # element.Tag
title_rows = title_block.findAll('tr') # element.ResultSet which is a list of Tags
title_links= [x.findAll('a') for x in title_rows] # list of list each with one element.Tag
title_data = [x[0].get_text()  for x in title_links if x]

DS_titles = [x.lower().replace(' ','+') for x in title_data if 'DATA SCI' in x]
DS_titles

['data+scientist',
 'senior+data+scientist',
 'associate+data+scientist',
 'data+scientist+ii',
 'data+scientist+i',
 'sr.+data+scientist',
 'lead+data+scientist',
 'data+science+engineer',
 'data+science+analyst',
 'senior+associate,+data+science']

In [6]:
# Manual Save Path Keywords based on DS_titles above
save_titles= ['DSgen','SeniorDS','AssocDS','DSII','DSI','SrDS','LeadDS','DSEng','DSAnalyst','SenAssocDS']

In [3]:
# Scrape Job Information and Pickle

for ind,title in enumerate(DS_titles):
    save_title = save_titles[ind]

    for year in years:
        h1b1_url = 'https://h1bdata.info/index.php?em=&job='+title+'&city=&year='+year

#         resp = requests.get(h1b1_url)

        h1b1_annual = BeautifulSoup(resp.content, 'html.parser')
        block = h1b1_annual.find('tbody')
        data = block.findAll('tr')
        t = [x.findAll('td') for x in data]

        col = ['Company','Role','Salary','Location','SubmitDate','StartDate','Status']
        ls = []
        for row in t:
            vals = [x.text for x in row]
            dic = dict(zip(col,vals))
            ls.append(dic)

        h1b1 = pd.DataFrame(ls)
        pause = random.randrange(3,15)
        print('Pausing for ',pause,' seconds...')
        time.sleep(pause)

        filename = 'data/h1b1data_'+save_title+'_'+year+'.p'
        outfile = open(filename,'wb')
        pickle.dump(h1b1,outfile)
        outfile.close()

In [69]:
# Load pickles and merge to one DataFrame

h1b1_all = pd.DataFrame()

for ind,title in enumerate(save_titles):
    
    for i, year in enumerate(years):
        
        filename = 'data/h1b1data_'+title+'_'+year+'.p'
        infile = open(filename,'rb')
        t = pickle.load(infile)
        t['Year'] = year

        if ((ind == 0) & (i == 0)):
            h1b1_all = t
        elif sum(t.shape)>=1:
            h1b1_all = pd.concat([h1b1_all,t], ignore_index=True, sort=True)
            

In [70]:
# Save to CSV
h1b1_all.to_csv('data/dirtydata.csv', index = False)